# **Data Collection and Preparation - Charging Sessions**

In [40]:
import pandas as pd
import numpy as np
import json

In [41]:
try:
    df = pd.read_csv('resources/charging_sessions.csv')
    print("Successfully imported charging_sessions.csv")
except FileNotFoundError:
    print("Error: charging_sessions.csv not found.")

Successfully imported charging_sessions.csv


In [42]:
df

,Unnamed: 0,id,connectionTime,disconnectTime,doneChargingTime,kWhDelivered,sessionID,siteID,spaceID,stationID,timezone,userID,userInputs
0,0,5e23b149f9af8b5fe4b973cf,2020-01-02 13:08:54+00:00,2020-01-02 19:11:15+00:00,2020-01-02 17:31:35+00:00,25.016,1_1_179_810_2020-01-02 13:08:53.870034,1,AG-3F30,1-1-179-810,America/Los_Angeles,194.0,"[{'WhPerMile': 250, 'kWhRequested': 25.0, 'mil..."
1,1,5e23b149f9af8b5fe4b973d0,2020-01-02 13:36:50+00:00,2020-01-02 22:38:21+00:00,2020-01-02 20:18:05+00:00,33.097,1_1_193_825_2020-01-02 13:36:49.599853,1,AG-1F01,1-1-193-825,America/Los_Angeles,4275.0,"[{'WhPerMile': 280, 'kWhRequested': 70.0, 'mil..."
2,2,5e23b149f9af8b5fe4b973d1,2020-01-02 13:56:35+00:00,2020-01-03 00:39:22+00:00,2020-01-02 16:35:06+00:00,6.521,1_1_193_829_2020-01-02 13:56:35.214993,1,AG-1F03,1-1-193-829,America/Los_Angeles,344.0,"[{'WhPerMile': 400, 'kWhRequested': 8.0, 'mile..."
3,3,5e23b149f9af8b5fe4b973d2,2020-01-02 13:59:58+00:00,2020-01-02 16:38:39+00:00,2020-01-02 15:18:45+00:00,2.355,1_1_193_820_2020-01-02 13:59:58.309319,1,AG-1F04,1-1-193-820,America/Los_Angeles,1117.0,"[{'WhPerMile': 400, 'kWhRequested': 8.0, 'mile..."
4,4,5e23b149f9af8b5fe4b973d3,2020-01-02 14:00:01+00:00,2020-01-02 22:08:40+00:00,2020-01-02 18:17:30+00:00,13.375,1_1_193_819_2020-01-02 14:00:00.779967,1,AG-1F06,1-1-193-819,America/Los_Angeles,334.0,"[{'WhPerMile': 400, 'kWhRequested': 16.0, 'mil..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
66445,10083,5d574ad2f9af8b4c10c03652,2019-07-31 18:08:04+00:00,2019-07-31 23:29:18+00:00,2019-07-31 23:30:18+00:00,28.787,1_1_179_809_2019-07-31 18:08:04.432654,1,AG-3F27,1-1-179-809,America/Los_Angeles,393.0,"[{'WhPerMile': 240, 'kWhRequested': 31.2, 'mil..."
66446,10084,5d574ad2f9af8b4c10c03653,2019-07-31 18:40:41+00:00,2019-08-01 00:59:42+00:00,2019-07-31 21:44:23+00:00,7.787,1_1_179_810_2019-07-31 18:40:40.900203,1,AG-3F30,1-1-179-810,America/Los_Angeles,220.0,"[{'WhPerMile': 333, 'kWhRequested': 6.66, 'mil..."
66447,10085,5d574ad2f9af8b4c10c03654,2019-07-31 19:04:40+00:00,2019-07-31 22:44:22+00:00,2019-07-31 22:45:21+00:00,11.274,1_1_191_795_2019-07-31 19:04:40.098273,1,AG-4F51,1-1-191-795,America/Los_Angeles,1974.0,"[{'WhPerMile': 333, 'kWhRequested': 19.98, 'mi..."
66448,10086,5d574ad2f9af8b4c10c03655,2019-07-31 19:19:47+00:00,2019-08-01 00:34:51+00:00,2019-07-31 21:25:30+00:00,11.589,1_1_191_778_2019-07-31 19:19:46.919358,1,AG-4F43,1-1-191-778,America/Los_Angeles,942.0,"[{'WhPerMile': 275, 'kWhRequested': 22.0, 'mil..."


### Format
We first observe that the .csv file's first column is an unnamed column with integers. Since there is no description in the .pdf file for the team assignment, we will drop these incoherent values and get rid of the very first column.

In [43]:
df = df.drop(df.columns[0], axis=1)

Furthermore, if we take a look at the ***userInputs*** column, we see that the provided data is in *json*-format.
For easier handling, we want to transform this column to an appropriate format for our analysis.
We will thus create a new column for each field in the json.

In [44]:
def clean_and_parse_json(value):
    if isinstance(value, str):
        cleaned_value = value.replace("'", '"')
        cleaned_value = cleaned_value.replace("True", "true").replace("False", "false")

        if cleaned_value.startswith("[") and cleaned_value.endswith("]"):
            try:
                return json.loads(cleaned_value)
            except json.JSONDecodeError as e:
                print(f"Error parsing JSON: {e}")
                return None
        else:
            return None
    return None


df['parsedJson'] = df['userInputs'].apply(clean_and_parse_json)


all_keys = set()
for row in df['parsedJson'].dropna():
    if isinstance(row, list) and row:
        all_keys.update(row[0].keys())

for field in all_keys:
    df[field] = df['parsedJson'].apply(
        lambda x: x[0].get(field) if isinstance(x, list) and x else np.nan
    )

df = df.drop(columns=['userInputs', 'parsedJson'])
df

,id,connectionTime,disconnectTime,doneChargingTime,kWhDelivered,sessionID,siteID,spaceID,stationID,timezone,userID,modifiedAt,milesRequested,paymentRequired,kWhRequested,requestedDeparture,minutesAvailable,WhPerMile
0,5e23b149f9af8b5fe4b973cf,2020-01-02 13:08:54+00:00,2020-01-02 19:11:15+00:00,2020-01-02 17:31:35+00:00,25.016,1_1_179_810_2020-01-02 13:08:53.870034,1,AG-3F30,1-1-179-810,America/Los_Angeles,194.0,"Thu, 02 Jan 2020 13:09:39 GMT",100.0,True,25.00,"Thu, 02 Jan 2020 20:51:54 GMT",463.0,250.0
1,5e23b149f9af8b5fe4b973d0,2020-01-02 13:36:50+00:00,2020-01-02 22:38:21+00:00,2020-01-02 20:18:05+00:00,33.097,1_1_193_825_2020-01-02 13:36:49.599853,1,AG-1F01,1-1-193-825,America/Los_Angeles,4275.0,"Thu, 02 Jan 2020 13:37:11 GMT",250.0,True,70.00,"Thu, 02 Jan 2020 23:31:50 GMT",595.0,280.0
2,5e23b149f9af8b5fe4b973d1,2020-01-02 13:56:35+00:00,2020-01-03 00:39:22+00:00,2020-01-02 16:35:06+00:00,6.521,1_1_193_829_2020-01-02 13:56:35.214993,1,AG-1F03,1-1-193-829,America/Los_Angeles,344.0,"Thu, 02 Jan 2020 13:57:17 GMT",20.0,True,8.00,"Thu, 02 Jan 2020 14:56:35 GMT",60.0,400.0
3,5e23b149f9af8b5fe4b973d2,2020-01-02 13:59:58+00:00,2020-01-02 16:38:39+00:00,2020-01-02 15:18:45+00:00,2.355,1_1_193_820_2020-01-02 13:59:58.309319,1,AG-1F04,1-1-193-820,America/Los_Angeles,1117.0,"Thu, 02 Jan 2020 14:00:03 GMT",20.0,True,8.00,"Thu, 02 Jan 2020 15:04:58 GMT",65.0,400.0
4,5e23b149f9af8b5fe4b973d3,2020-01-02 14:00:01+00:00,2020-01-02 22:08:40+00:00,2020-01-02 18:17:30+00:00,13.375,1_1_193_819_2020-01-02 14:00:00.779967,1,AG-1F06,1-1-193-819,America/Los_Angeles,334.0,"Thu, 02 Jan 2020 14:00:13 GMT",40.0,True,16.00,"Thu, 02 Jan 2020 22:24:01 GMT",504.0,400.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66445,5d574ad2f9af8b4c10c03652,2019-07-31 18:08:04+00:00,2019-07-31 23:29:18+00:00,2019-07-31 23:30:18+00:00,28.787,1_1_179_809_2019-07-31 18:08:04.432654,1,AG-3F27,1-1-179-809,America/Los_Angeles,393.0,"Wed, 31 Jul 2019 18:08:23 GMT",130.0,True,31.20,"Thu, 01 Aug 2019 00:03:04 GMT",355.0,240.0
66446,5d574ad2f9af8b4c10c03653,2019-07-31 18:40:41+00:00,2019-08-01 00:59:42+00:00,2019-07-31 21:44:23+00:00,7.787,1_1_179_810_2019-07-31 18:40:40.900203,1,AG-3F30,1-1-179-810,America/Los_Angeles,220.0,"Wed, 31 Jul 2019 18:41:02 GMT",20.0,True,6.66,"Thu, 01 Aug 2019 02:15:41 GMT",455.0,333.0
66447,5d574ad2f9af8b4c10c03654,2019-07-31 19:04:40+00:00,2019-07-31 22:44:22+00:00,2019-07-31 22:45:21+00:00,11.274,1_1_191_795_2019-07-31 19:04:40.098273,1,AG-4F51,1-1-191-795,America/Los_Angeles,1974.0,"Wed, 31 Jul 2019 19:04:57 GMT",60.0,True,19.98,"Wed, 31 Jul 2019 22:08:40 GMT",184.0,333.0
66448,5d574ad2f9af8b4c10c03655,2019-07-31 19:19:47+00:00,2019-08-01 00:34:51+00:00,2019-07-31 21:25:30+00:00,11.589,1_1_191_778_2019-07-31 19:19:46.919358,1,AG-4F43,1-1-191-778,America/Los_Angeles,942.0,"Wed, 31 Jul 2019 19:20:10 GMT",80.0,True,22.00,"Wed, 31 Jul 2019 20:19:47 GMT",60.0,275.0


Now we take a look at the datatypes in our dataframe:

In [45]:
df.dtypes

id                     object
connectionTime         object
disconnectTime         object
doneChargingTime       object
kWhDelivered          float64
sessionID              object
siteID                  int64
spaceID                object
stationID              object
timezone               object
userID                float64
modifiedAt             object
milesRequested        float64
paymentRequired        object
kWhRequested          float64
requestedDeparture     object
minutesAvailable      float64
WhPerMile             float64
dtype: object

For easier handling during our analysis, it might be benificial to convert the values of **connectionTime**, **disconnectTime**, **doneChargingtime**, **modifiedAt** and **requestedDeparture** from type object to datetime. Further, we know by the supplemented document of the team assignment that all datetimes are in UTC. For easier use, we will convert all datetimes from Timezone *UTC* to *America/Los Angeles*.

In [46]:
df['connectionTime'] = pd.to_datetime(df['connectionTime'], utc=True, errors='coerce').dt.tz_convert('America/Los_Angeles')
df['disconnectTime'] = pd.to_datetime(df['disconnectTime'], utc=True, errors='coerce').dt.tz_convert('America/Los_Angeles')
df['doneChargingTime'] = pd.to_datetime(df['doneChargingTime'], utc=True, errors='coerce').dt.tz_convert('America/Los_Angeles')
df['modifiedAt'] = pd.to_datetime(df['modifiedAt'], utc=True, errors='coerce').dt.tz_convert('America/Los_Angeles')
df['requestedDeparture'] = pd.to_datetime(df['requestedDeparture'], utc=True, errors='coerce').dt.tz_convert('America/Los_Angeles')
df.dtypes

id                                                 object
connectionTime        datetime64[ns, America/Los_Angeles]
disconnectTime        datetime64[ns, America/Los_Angeles]
doneChargingTime      datetime64[ns, America/Los_Angeles]
kWhDelivered                                      float64
sessionID                                          object
siteID                                              int64
spaceID                                            object
stationID                                          object
timezone                                           object
userID                                            float64
modifiedAt            datetime64[ns, America/Los_Angeles]
milesRequested                                    float64
paymentRequired                                    object
kWhRequested                                      float64
requestedDeparture    datetime64[ns, America/Los_Angeles]
minutesAvailable                                  float64
WhPerMile     

In [47]:
df.head(3)

,id,connectionTime,disconnectTime,doneChargingTime,kWhDelivered,sessionID,siteID,spaceID,stationID,timezone,userID,modifiedAt,milesRequested,paymentRequired,kWhRequested,requestedDeparture,minutesAvailable,WhPerMile
0,5e23b149f9af8b5fe4b973cf,2020-01-02 05:08:54-08:00,2020-01-02 11:11:15-08:00,2020-01-02 09:31:35-08:00,25.016,1_1_179_810_2020-01-02 13:08:53.870034,1,AG-3F30,1-1-179-810,America/Los_Angeles,194.0,2020-01-02 05:09:39-08:00,100.0,True,25.0,2020-01-02 12:51:54-08:00,463.0,250.0
1,5e23b149f9af8b5fe4b973d0,2020-01-02 05:36:50-08:00,2020-01-02 14:38:21-08:00,2020-01-02 12:18:05-08:00,33.097,1_1_193_825_2020-01-02 13:36:49.599853,1,AG-1F01,1-1-193-825,America/Los_Angeles,4275.0,2020-01-02 05:37:11-08:00,250.0,True,70.0,2020-01-02 15:31:50-08:00,595.0,280.0
2,5e23b149f9af8b5fe4b973d1,2020-01-02 05:56:35-08:00,2020-01-02 16:39:22-08:00,2020-01-02 08:35:06-08:00,6.521,1_1_193_829_2020-01-02 13:56:35.214993,1,AG-1F03,1-1-193-829,America/Los_Angeles,344.0,2020-01-02 05:57:17-08:00,20.0,True,8.0,2020-01-02 06:56:35-08:00,60.0,400.0


Let's also take a look at the amount of distinct values for each column.

In [48]:
df.nunique(dropna=True)

id                    65037
connectionTime        64839
disconnectTime        64906
doneChargingTime      60637
kWhDelivered          25629
sessionID             65037
siteID                    2
spaceID                 107
stationID               107
timezone                  1
userID                 1006
modifiedAt            47704
milesRequested          106
paymentRequired           1
kWhRequested           1201
requestedDeparture    47773
minutesAvailable        842
WhPerMile               178
dtype: int64

We observe that for the columns **timezone** and **paymentRequired**, there is only **one** unique value which is not null.
Thus, we can drop them as they don't add any value to our analysis since they can't be used to compare / differentiate rows.

In [49]:
df = df.drop(columns=['timezone','paymentRequired'])

Further, we see that there is the same amount of spaceIDs as stationIDs (both 107). Let's take a look at how many different combinations there are.

In [50]:
df.drop_duplicates(subset=['stationID', 'spaceID']).shape[0]

107

We observe that there are also only 107 possible combinations. This means that there is a 1:1 mapping between stationIDs and spaceIDs. Keeping both is unnecessary as the information would be redundant, thus we will drop the *spaceID*.

In [51]:
df = df.drop(columns='spaceID', axis=1)

### Duplicates
First we want to find out whether any duplicates exist.
If so, those duplicates shall be removed.

In [52]:
len(df[df.duplicated()])

1413

As we can see, there are **1413** duplicated rows. We will thus remove them for further analysis.

In [53]:
df = df.drop_duplicates()

### Missing Data
We now want to check the data for missing values

In [54]:
df.isnull().sum()

id                        0
connectionTime            0
disconnectTime            0
doneChargingTime       4087
kWhDelivered              0
sessionID                 0
siteID                    0
stationID                 0
userID                17215
modifiedAt            17215
milesRequested        17215
kWhRequested          17215
requestedDeparture    17215
minutesAvailable      17215
WhPerMile             17215
dtype: int64

As we can see, there are **4087** rows with a missing **doneChargingTime** value as well as **17215** rows with missing **userID** and **userInputs** value. The latter is expected as charging sessions may also be carried out by unregistered users. Thus, we only have to take care of the **doneChargingTime** values. We try to find out whether this is related to a specific other value.

In [55]:
df[df.doneChargingTime.isnull()]['stationID'].value_counts()

stationID
2-39-81-4550    1125
1-1-178-823       89
1-1-193-820       77
1-1-178-828       67
1-1-193-827       63
                ... 
2-39-129-563       2
2-39-126-560       1
2-39-125-559       1
2-39-130-564       1
2-39-82-384        1
Name: count, Length: 100, dtype: int64

The stationID **2-39-81-4550** seems to be faulty when it comes to this value. When we look at its records:

In [56]:
print("stationID 2-39-81-4550")
print(f"non null values: {df[df.stationID == '2-39-81-4550']['doneChargingTime'].notnull().sum()}")

stationID 2-39-81-4550
non null values: 0


We see that this station is always missing the **doneChargingTime** value.

## Invalid Data

Now we also want to check for records that are invalid (and don't make any sense).

In [57]:
print(f"Amount of records with disconnectTime before connectionTime: {len(df[df.connectionTime > df.disconnectTime])}")
print(f"Amount of records with doneChargingTime before connectionTime: {len(df[df.connectionTime > df.doneChargingTime])}")
print(f"Amount of records with negative kWhDelivered: {len(df[df.kWhDelivered < 0])}")

Amount of records with disconnectTime before connectionTime: 0
Amount of records with doneChargingTime before connectionTime: 27
Amount of records with negative kWhDelivered: 0


## New features

We will also add some new features which we consider useful for our further analysis

In [58]:
df['connectionDuration'] = df['disconnectTime'] - df['connectionTime']
df['chargingDuration'] = df['doneChargingTime'] - df['connectionTime']
df['idleDuration'] = abs(df['chargingDuration'] - df['connectionDuration'])
df['kWh_diff'] = np.where(
    df['userID'].notnull(),
    df['kWhRequested'] - df['kWhDelivered'],
    np.nan
)
df

,id,connectionTime,disconnectTime,doneChargingTime,kWhDelivered,sessionID,siteID,stationID,userID,modifiedAt,milesRequested,kWhRequested,requestedDeparture,minutesAvailable,WhPerMile,connectionDuration,chargingDuration,idleDuration,kWh_diff
0,5e23b149f9af8b5fe4b973cf,2020-01-02 05:08:54-08:00,2020-01-02 11:11:15-08:00,2020-01-02 09:31:35-08:00,25.016,1_1_179_810_2020-01-02 13:08:53.870034,1,1-1-179-810,194.0,2020-01-02 05:09:39-08:00,100.0,25.00,2020-01-02 12:51:54-08:00,463.0,250.0,0 days 06:02:21,0 days 04:22:41,0 days 01:39:40,-0.016
1,5e23b149f9af8b5fe4b973d0,2020-01-02 05:36:50-08:00,2020-01-02 14:38:21-08:00,2020-01-02 12:18:05-08:00,33.097,1_1_193_825_2020-01-02 13:36:49.599853,1,1-1-193-825,4275.0,2020-01-02 05:37:11-08:00,250.0,70.00,2020-01-02 15:31:50-08:00,595.0,280.0,0 days 09:01:31,0 days 06:41:15,0 days 02:20:16,36.903
2,5e23b149f9af8b5fe4b973d1,2020-01-02 05:56:35-08:00,2020-01-02 16:39:22-08:00,2020-01-02 08:35:06-08:00,6.521,1_1_193_829_2020-01-02 13:56:35.214993,1,1-1-193-829,344.0,2020-01-02 05:57:17-08:00,20.0,8.00,2020-01-02 06:56:35-08:00,60.0,400.0,0 days 10:42:47,0 days 02:38:31,0 days 08:04:16,1.479
3,5e23b149f9af8b5fe4b973d2,2020-01-02 05:59:58-08:00,2020-01-02 08:38:39-08:00,2020-01-02 07:18:45-08:00,2.355,1_1_193_820_2020-01-02 13:59:58.309319,1,1-1-193-820,1117.0,2020-01-02 06:00:03-08:00,20.0,8.00,2020-01-02 07:04:58-08:00,65.0,400.0,0 days 02:38:41,0 days 01:18:47,0 days 01:19:54,5.645
4,5e23b149f9af8b5fe4b973d3,2020-01-02 06:00:01-08:00,2020-01-02 14:08:40-08:00,2020-01-02 10:17:30-08:00,13.375,1_1_193_819_2020-01-02 14:00:00.779967,1,1-1-193-819,334.0,2020-01-02 06:00:13-08:00,40.0,16.00,2020-01-02 14:24:01-08:00,504.0,400.0,0 days 08:08:39,0 days 04:17:29,0 days 03:51:10,2.625
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65032,5d2fbdd3f9af8b4d0dd0d54a,2019-07-01 14:49:12-07:00,2019-07-01 18:37:18-07:00,2019-07-01 16:14:33-07:00,8.399,1_1_179_798_2019-07-01 21:49:11.873404,1,1-1-179-798,1346.0,2019-07-01 14:49:37-07:00,80.0,32.00,2019-07-01 15:25:12-07:00,36.0,400.0,0 days 03:48:06,0 days 01:25:21,0 days 02:22:45,23.601
65033,5d2fbdd3f9af8b4d0dd0d54b,2019-07-01 14:58:45-07:00,2019-07-01 17:39:48-07:00,2019-07-01 17:40:21-07:00,16.864,1_1_179_794_2019-07-01 21:58:44.571011,1,1-1-179-794,364.0,2019-07-01 14:59:09-07:00,100.0,40.00,2019-07-01 18:52:45-07:00,234.0,400.0,0 days 02:41:03,0 days 02:41:36,0 days 00:00:33,23.136
65034,5d2fbdd3f9af8b4d0dd0d54c,2019-07-01 15:02:21-07:00,2019-07-01 17:58:50-07:00,2019-07-01 17:59:23-07:00,18.335,1_1_191_807_2019-07-01 22:02:20.810735,1,1-1-191-807,2050.0,2019-07-01 15:02:37-07:00,90.0,29.97,2019-07-01 20:11:21-07:00,309.0,333.0,0 days 02:56:29,0 days 02:57:02,0 days 00:00:33,11.635
65035,5d2fbdd3f9af8b4d0dd0d54d,2019-07-01 15:23:44-07:00,2019-07-01 19:03:36-07:00,2019-07-01 19:04:06-07:00,22.815,1_1_179_781_2019-07-01 22:23:32.496137,1,1-1-179-781,1626.0,2019-07-01 15:23:43-07:00,170.0,34.00,2019-07-01 17:02:44-07:00,99.0,200.0,0 days 03:39:52,0 days 03:40:22,0 days 00:00:30,11.185


In [59]:
df.to_csv('resources/charging_sessions_prepared.csv', index=False)